In [49]:
from pathlib import Path

import h5py
import numpy as np
import torch

dataset_dir = Path("../data/interim/coco/2subbins")
number_of_images_to_generate_per_input = 2001
number_of_temporal_bins = 100
h5_compression = 3

In [50]:
index = 1

directory_list = [
    path for path in dataset_dir.glob("**/*") if path.is_dir()
]

video_path = directory_list[index] / "ground_truth.h5"
with h5py.File(video_path, "r") as file:
    video = np.array(file["synchronized_video"])

input_path = directory_list[index] / "input.h5"
with h5py.File(input_path, "r") as file:
    generated_video = np.array(file["generated_video"])


In [51]:
assert (
    number_of_images_to_generate_per_input - 1
) % number_of_temporal_bins == 0

discretized_frame_length = (
    number_of_images_to_generate_per_input - 1
) // number_of_temporal_bins

indices = torch.arange(
    discretized_frame_length,
    number_of_images_to_generate_per_input,
    discretized_frame_length,
    dtype=torch.int64,
)
new_synchronized_video = generated_video[indices]

In [52]:
with h5py.File(video_path, "w") as file:
    file.create_dataset(
        "/synchronized_video",
        data=new_synchronized_video,
        compression="gzip",
        compression_opts=h5_compression,
    )
